In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import vice
import seaborn as sns

from statsmodels.distributions import ECDF

In [ ]:
import surp
from surp import ViceModel

import arya
arya.style.set_size((10/3, 10/3))

In [ ]:
giants = pd.read_csv(surp.DATA_DIR + "apogee_cartography.csv", index_col=0)

In [ ]:
# Grab cdf
cdf = ECDF(giants.R_gal)

bins = np.arange(0, 20, 0.1)

zones = np.arange(0, 200)
Rs = (0.5 + zones)/10
table = pd.DataFrame({"zone": zones, "R": Rs, "cdf": cdf(bins + 0.1)/cdf(bins + 0.1)[-1]})

In [ ]:
fiducial = ViceModel.from_file("../../models/C11_f0.2/model.json").stars_unsampled

In [ ]:
fiducial.high_alpha

# Create matching sample

In [ ]:
bins

In [ ]:
num = len(giants)
num

In [ ]:

sample = pd.DataFrame()
for _ in range(num):
    star = surp.vice_utils.rand_star(fiducial, table, width=0.1)
    sample = pd.concat((sample, star), ignore_index=True)

In [ ]:
sns.kdeplot(sample.r_final, label="model")
sns.kdeplot(giants.R_gal, label="data")

In [ ]:
sns.kdeplot(sample[~sample.high_alpha].r_final, label="model")
sns.kdeplot(giants[~giants.high_alpha].R_gal, label="data")

In [ ]:
sample.high_alpha = sample.high_alpha.convert_dtypes()

In [ ]:
surp.yields.set_magg22_scale()

In [ ]:
giants["CPN_MG"] = giants["CPN_H"] - giants.MG_H

In [ ]:
sample["C_H"] = sample.C_FE + sample.FE_H
sample["N_H"] = sample.N_FE + sample.FE_H
sample["CPN_H"] = surp.gce_math.cpn(sample.C_H, sample.N_H)
sample["CPN_MG"] = sample.CPN_H - sample.MG_H

In [ ]:
w22_x = [-0.6877142487549317, -0.5943260461807127, -0.49556949744518464, -0.403644654291443, -0.2991155164607723, -0.1990734751956536, -0.09569238729707008, 0.00743806998253671, 0.11101319448936031, 0.196316538386909, 0.3030043334842507, 0.40380635146497634]
w22_y = [-0.1377369846524225, -0.07126090881733371, -0.06708396027685826, -0.07602768582606079, -0.06916039723141743, -0.05035811014143843, -0.025880228708997866, -0.0060186578393017776, 0.027469154378573557, 0.0502196810111345, 0.06622329220583811, 0.08085464941318088]

In [ ]:
df = giants[giants["high_alpha"]].copy()

arya.medianplot(df, x="MG_H", y="CPN_MG", numbins=20, stat="median", errorbar="pi", label="APOGEE Giants")


df = giants[~giants["high_alpha"]].copy()

arya.medianplot(df, x="MG_H", y="CPN_MG", numbins=20, stat="median", errorbar="pi", label="APOGEE Giants")


In [ ]:
np.log10(vice.solar_z("c")/12) + 12 # versus 8.39

In [ ]:
np.log10(vice.solar_z("n")/14) + 12 # versus 7.78

In [ ]:
surp.gce_math.cpn(8.45 - 8.39, 7.78 - 7.87)

In [ ]:
df = giants[~giants["high_alpha"]].copy()

arya.medianplot(df, x="MG_H", y="CPN_MG", numbins=20, stat="median", errorbar="pi", label="APOGEE Giants")

df = sample[~sample.high_alpha].copy()
df["CPN_MG"] += 0.08

arya.medianplot(df, x="MG_H", y="CPN_MG", stat="median", errorbar=None,
    aes="line", numbins=30, label="model", color=arya.COLORS[1])

                                        
plt.xlabel("[Mg/H]")
plt.ylabel("[C+N/Mg]")
plt.xlim(-0.7)

#plt.scatter(w22_x, np.array(w22_y) + 0.029, color=arya.COLORS[2])

arya.Legend(color_only=True)

plt.savefig("c+n.pdf")

# Subgiants

In [ ]:
sample_sg = ViceModel.from_file("../../models/C11_f0.2/model.json").stars

In [ ]:
def add_cpn(df):
    df["C_H"] = df.C_FE + df.FE_H
    df["N_H"] = df.N_FE + df.FE_H
    df["CPN_H"] = surp.gce_math.cpn(df.C_H, df.N_H)
    df["CPN_MG"] = df.CPN_H - df.MG_H

In [ ]:
add_cpn(sample_sg)

In [ ]:
subgiants = surp.subgiants

In [ ]:
add_cpn(subgiants)

In [ ]:
df = subgiants[~subgiants["high_alpha"]].copy()

arya.medianplot(df, x="MG_H", y="CPN_MG", numbins=20, stat="median", errorbar="pi", label="APOGEE subgiants")

df = sample_sg[~sample_sg.high_alpha].copy()

arya.medianplot(df, x="MG_H", y="CPN_MG", stat="median", errorbar=None,
    aes="line", numbins=30, label="model", color=arya.COLORS[1])

                                        
plt.xlabel("[Mg/H]")
plt.ylabel("[C+N/Mg]")
plt.xlim(-0.7)

#plt.scatter(w22_x, np.array(w22_y) + 0.029, color=arya.COLORS[2])

arya.Legend(color_only=True)
